# Análise de Temperatura por Região do Brasil

Este notebook realiza uma análise detalhada das temperaturas nas diferentes regiões do Brasil, utilizando dados do INMET de 2024.

In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from glob import glob
import os

# Configurações de visualização
plt.style.use('seaborn')
sns.set_theme(style="whitegrid")
%matplotlib inline

# Configurando o pandas para exibir mais colunas
pd.set_option('display.max_columns', None)

## Carregamento e Preparação dos Dados

Vamos carregar os dados de todas as estações meteorológicas e organizá-los por região.

In [ ]:
def carregar_dados_regiao(pasta_dados, regiao):
    """Carrega dados de uma região específica.
    
    Args:
        pasta_dados (str): Caminho para a pasta com os arquivos CSV
        regiao (str): Sigla da região (N, NE, CO, SE, S)
        
    Returns:
        pd.DataFrame: DataFrame com os dados da região
    """
    arquivos = glob(os.path.join(pasta_dados, f'INMET_{regiao}_*.CSV'))
    dfs = []
    
    for arquivo in arquivos:
        try:
            df = pd.read_csv(arquivo, sep=';', decimal=',')
            df['REGIAO'] = regiao
            df['ESTACAO'] = os.path.basename(arquivo).split('_')[3]
            dfs.append(df)
        except Exception as e:
            print(f'Erro ao ler {arquivo}: {e}')
            
    return pd.concat(dfs, ignore_index=True) if dfs else None

# Carregar dados de todas as regiões
pasta_dados = '../data'
regioes = ['N', 'NE', 'CO', 'SE', 'S']
dados_por_regiao = {}

for regiao in regioes:
    dados_por_regiao[regiao] = carregar_dados_regiao(pasta_dados, regiao)

## Análise de Temperatura por Região

Vamos analisar as temperaturas médias, máximas e mínimas por região.

In [ ]:
def analise_temperatura_regiao(df, regiao):
    """Realiza análise estatística das temperaturas de uma região.
    
    Args:
        df (pd.DataFrame): DataFrame com os dados
        regiao (str): Nome da região
    """
    if df is None:
        print(f'Sem dados para a região {regiao}')
        return
    
    # Estatísticas básicas
    stats = df[['TEMPERATURA MÁXIMA, (°C)', 'TEMPERATURA MÍNIMA, (°C)', 'TEMPERATURA MÉDIA, (°C)']].describe()
    
    # Plotar distribuição das temperaturas
    plt.figure(figsize=(15, 6))
    sns.boxplot(data=df[['TEMPERATURA MÁXIMA, (°C)', 'TEMPERATURA MÍNIMA, (°C)', 'TEMPERATURA MÉDIA, (°C)']])
    plt.title(f'Distribuição de Temperaturas - Região {regiao}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'../reports/temp_distribuicao_{regiao}.png')
    plt.close()
    
    return stats

# Analisar cada região
for regiao, dados in dados_por_regiao.items():
    print(f'\nAnálise da Região {regiao}')
    stats = analise_temperatura_regiao(dados, regiao)
    if stats is not None:
        print(stats)

## Análise Temporal das Temperaturas

Vamos analisar como as temperaturas variam ao longo do tempo em cada região.

In [ ]:
def analise_temporal(df, regiao):
    """Realiza análise temporal das temperaturas.
    
    Args:
        df (pd.DataFrame): DataFrame com os dados
        regiao (str): Nome da região
    """
    if df is None:
        return
    
    # Converter data para datetime
    df['DATA'] = pd.to_datetime(df['DATA'])
    
    # Média diária por região
    medias_diarias = df.groupby('DATA')[['TEMPERATURA MÁXIMA, (°C)', 
                                         'TEMPERATURA MÍNIMA, (°C)', 
                                         'TEMPERATURA MÉDIA, (°C)']].mean()
    
    # Plotar série temporal
    plt.figure(figsize=(15, 6))
    plt.plot(medias_diarias.index, medias_diarias['TEMPERATURA MÁXIMA, (°C)'], label='Máxima')
    plt.plot(medias_diarias.index, medias_diarias['TEMPERATURA MÍNIMA, (°C)'], label='Mínima')
    plt.plot(medias_diarias.index, medias_diarias['TEMPERATURA MÉDIA, (°C)'], label='Média')
    plt.title(f'Variação de Temperatura ao Longo do Tempo - Região {regiao}')
    plt.xlabel('Data')
    plt.ylabel('Temperatura (°C)')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'../reports/temp_temporal_{regiao}.png')
    plt.close()

# Análise temporal para cada região
for regiao, dados in dados_por_regiao.items():
    analise_temporal(dados, regiao)

## Comparação Entre Regiões

Vamos criar visualizações que comparam as temperaturas entre as diferentes regiões do Brasil.

In [ ]:
def comparar_regioes(dados_por_regiao):
    """Cria visualizações comparativas entre as regiões.
    
    Args:
        dados_por_regiao (dict): Dicionário com DataFrames por região
    """
    # Preparar dados para comparação
    dados_comparacao = []
    for regiao, df in dados_por_regiao.items():
        if df is not None:
            medias = df[['TEMPERATURA MÁXIMA, (°C)', 'TEMPERATURA MÍNIMA, (°C)', 'TEMPERATURA MÉDIA, (°C)']].mean()
            dados_comparacao.append({
                'Região': regiao,
                'Temperatura Máxima': medias['TEMPERATURA MÁXIMA, (°C)'],
                'Temperatura Mínima': medias['TEMPERATURA MÍNIMA, (°C)'],
                'Temperatura Média': medias['TEMPERATURA MÉDIA, (°C)']
            })
    
    df_comparacao = pd.DataFrame(dados_comparacao)
    
    # Criar gráfico de barras comparativo
    plt.figure(figsize=(12, 6))
    x = np.arange(len(df_comparacao['Região']))
    width = 0.25
    
    plt.bar(x - width, df_comparacao['Temperatura Máxima'], width, label='Máxima')
    plt.bar(x, df_comparacao['Temperatura Média'], width, label='Média')
    plt.bar(x + width, df_comparacao['Temperatura Mínima'], width, label='Mínima')
    
    plt.xlabel('Região')
    plt.ylabel('Temperatura (°C)')
    plt.title('Comparação de Temperaturas entre Regiões')
    plt.xticks(x, df_comparacao['Região'])
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('../reports/temp_comparacao_regioes.png')
    plt.close()

# Realizar comparação entre regiões
comparar_regioes(dados_por_regiao)